In [121]:
import numpy as np
import pandas as pd
import datetime
import math


In [122]:
dam = pd.read_csv('dam.csv')
dam

,hourlyid,market,tradedate,hour,price,volume
0,26345,1,2016-01-01,0,40.10,477.5
1,26346,1,2016-01-01,1,40.08,486.6
2,26347,1,2016-01-01,2,40.06,495.6
3,26348,1,2016-01-01,3,40.08,496.6
4,26349,1,2016-01-01,4,40.07,502.9
...,...,...,...,...,...,...
46723,73029,1,2021-04-29,4,39.97,126.1
46724,73037,1,2021-04-29,12,68.06,71.0
46725,73049,1,2021-04-30,0,44.04,59.6
46726,73057,1,2021-04-30,8,190.00,72.0


In [123]:
dam['date']=pd.to_datetime(dam['tradedate'],format="%Y/%m/%d")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100

dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['day_of_week']>4 else 'weekday_' + str(x['hour'])),axis=1)
dam=dam.drop(['hourlyid','tradedate','date'],axis=1)
dam

,market,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,
2016-01-01,1,0,40.10,477.5,2016,1,1,4,2016.01,weekday_0
2016-01-01,1,1,40.08,486.6,2016,1,1,4,2016.01,weekday_1
2016-01-01,1,2,40.06,495.6,2016,1,1,4,2016.01,weekday_2
2016-01-01,1,3,40.08,496.6,2016,1,1,4,2016.01,weekday_3
2016-01-01,1,4,40.07,502.9,2016,1,1,4,2016.01,weekday_4
...,...,...,...,...,...,...,...,...,...,...
2021-04-29,1,4,39.97,126.1,2021,4,29,3,2021.04,weekday_4
2021-04-29,1,12,68.06,71.0,2021,4,29,3,2021.04,weekday_12
2021-04-30,1,0,44.04,59.6,2021,4,30,4,2021.04,weekday_0


In [124]:
dam.loc[dam['hour']<12].groupby(['day','month','year']).max()['price']


day  month  year
1    1      2016     48.03
            2017     33.47
            2018     34.16
            2019     27.26
            2020     80.95
                     ...  
31   12     2016     53.28
            2017     31.57
            2018     56.80
            2019     88.52
            2020    102.37
Name: price, Length: 1947, dtype: float64

In [125]:
dam['value']=dam['price']*dam['volume']
daily=dam.groupby(['year','month','day']).sum().drop(['hour','day_of_week','month_no','market'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=dam[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=dam[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=dam[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['price_morning_max']=dam.loc[dam['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=dam.loc[dam['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=dam.loc[(dam['hour']>=12) & (dam['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 4     26   2092.9  185605.239        88.683281     299.33      39.94   
           27   2048.2  168261.497        82.150912     299.09      39.96   
           28   2130.2  177543.993        83.346161     299.10      39.90   
           29   2134.0  178697.984        83.738512     296.82      39.79   
           30   2198.9  187057.475        85.068659     295.98      38.97   

                price_mean  price_morning_max  price_evening_max  \
year month day                                                     
2016 1     1     45.982500              48.03              60.08   
           2     54.114167              56.05             118.49   
           3     54.959167              56.01             118.47   
           4     80.164583              80.02             177.66   
           5     84.503333             148.40             181.40   
...                    ...                ...                ...   
2021 4     26   102.215000             201.07             299.33   
           27    94.468333             201.03             299.09   
           28   100.343333             198.58             299.10   
           29    98.742500             195.01             296.82   
           30    98.218750             190.00             295.98   

                price_standard  
year month day                  
2016 1     1           45.8275  
           2           50.2275  
           3           50.6050  
           4           80.0200  
           5           80.0500  
...                        ...  
2021 4     26          70.0450  
           27          70.0200  
           28          70.0125  
           29          68.0600  
           30          67.9825  

[1947 rows x 9 columns]

In [126]:
dam=dam.reset_index().set_index(['year','month','day'])
dam

date  market  hour   price  volume  day_of_week  \
year month day                                                         
2016 1     1   2016-01-01       1     0   40.10   477.5            4   
           1   2016-01-01       1     1   40.08   486.6            4   
           1   2016-01-01       1     2   40.06   495.6            4   
           1   2016-01-01       1     3   40.08   496.6            4   
           1   2016-01-01       1     4   40.07   502.9            4   
...                   ...     ...   ...     ...     ...          ...   
2021 4     29  2021-04-29       1     4   39.97   126.1            3   
           29  2021-04-29       1    12   68.06    71.0            3   
           30  2021-04-30       1     0   44.04    59.6            4   
           30  2021-04-30       1     8  190.00    72.0            4   
           30  2021-04-30       1    17   67.97   114.2            4   

                month_no    hour_ref      value  
year month day                                   
2016 1     1     2016.01   weekday_0  19147.750  
           1     2016.01   weekday_1  19502.928  
           1     2016.01   weekday_2  19853.736  
           1     2016.01   weekday_3  19903.728  
           1     2016.01   weekday_4  20151.203  
...                  ...         ...        ...  
2021 4     29    2021.04   weekday_4   5040.217  
           29    2021.04  weekday_12   4832.260  
           30    2021.04   weekday_0   2624.784  
           30    2021.04   weekday_8  13680.000  
           30    2021.04  weekday_17   7762.174  

[46728 rows x 9 columns]

In [127]:

dam['price_min']=daily['price_min']
dam['price_max']=daily['price_max']
dam['price_mean']=daily['price_mean']
dam['price_standard']=daily['price_standard']

dam['price_morning_max']=daily['price_morning_max']
dam['price_evening_max']=daily['price_evening_max']


dam['cat']='standard'
dam.loc[(dam['hour']<11) & (dam['day_of_week']<5) & (dam['price_morning_max'] > dam['price_standard'] * 1.3) & (dam['price'] > dam['price_morning_max']-(dam['price_mean'] * 0.2)),'cat']='morning'
dam.loc[(dam['hour']>=11) & (dam['day_of_week']<5) & (dam['price_evening_max'] > dam['price_standard'] * 1.3) & (dam['price'] > dam['price_evening_max']-(dam['price_mean'] * 0.2)),'cat']='evening'

dam.loc[dam['price'] < dam['price_min']+(dam['price_mean'] * 0.2),'cat']='off'

dam.loc[dam['price_max'] - dam['price_min']<30,'cat']='off'
dam

date  market  hour   price  volume  day_of_week  \
year month day                                                         
2016 1     1   2016-01-01       1     0   40.10   477.5            4   
           1   2016-01-01       1     1   40.08   486.6            4   
           1   2016-01-01       1     2   40.06   495.6            4   
           1   2016-01-01       1     3   40.08   496.6            4   
           1   2016-01-01       1     4   40.07   502.9            4   
...                   ...     ...   ...     ...     ...          ...   
2021 4     29  2021-04-29       1     4   39.97   126.1            3   
           29  2021-04-29       1    12   68.06    71.0            3   
           30  2021-04-30       1     0   44.04    59.6            4   
           30  2021-04-30       1     8  190.00    72.0            4   
           30  2021-04-30       1    17   67.97   114.2            4   

                month_no    hour_ref      value  price_min  price_max  \
year month day                                                          
2016 1     1     2016.01   weekday_0  19147.750      35.09      60.08   
           1     2016.01   weekday_1  19502.928      35.09      60.08   
           1     2016.01   weekday_2  19853.736      35.09      60.08   
           1     2016.01   weekday_3  19903.728      35.09      60.08   
           1     2016.01   weekday_4  20151.203      35.09      60.08   
...                  ...         ...        ...        ...        ...   
2021 4     29    2021.04   weekday_4   5040.217      39.79     296.82   
           29    2021.04  weekday_12   4832.260      39.79     296.82   
           30    2021.04   weekday_0   2624.784      38.97     295.98   
           30    2021.04   weekday_8  13680.000      38.97     295.98   
           30    2021.04  weekday_17   7762.174      38.97     295.98   

                price_mean  price_standard  price_morning_max  \
year month day                                                  
2016 1     1      45.98250         45.8275              48.03   
           1      45.98250         45.8275              48.03   
           1      45.98250         45.8275              48.03   
           1      45.98250         45.8275              48.03   
           1      45.98250         45.8275              48.03   
...                    ...             ...                ...   
2021 4     29     98.74250         68.0600             195.01   
           29     98.74250         68.0600             195.01   
           30     98.21875         67.9825             190.00   
           30     98.21875         67.9825             190.00   
           30     98.21875         67.9825             190.00   

                price_evening_max       cat  
year month day                               
2016 1     1                60.08       off  
           1                60.08       off  
           1                60.08       off  
           1                60.08       off  
           1                60.08       off  
...                           ...       ...  
2021 4     29              296.82       off  
           29              296.82  standard  
           30              295.98       off  
           30              295.98   morning  
           30              295.98  standard  

[46728 rows x 16 columns]

In [128]:
daily['off_price_mean']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=dam.loc[dam['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=dam.loc[dam['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=dam.loc[dam['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=dam.loc[dam['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

#daily=daily.drop(['price_min','price_max','price_morning_max','price_evening_max'], axis=1)

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2021 4     26   2092.9  185605.239        88.683281     299.33      39.94   
           27   2048.2  168261.497        82.150912     299.09      39.96   
           28   2130.2  177543.993        83.346161     299.10      39.90   
           29   2134.0  178697.984        83.738512     296.82      39.79   
           30   2198.9  187057.475        85.068659     295.98      38.97   

                price_mean  price_morning_max  price_evening_max  \
year month day                                                     
2016 1     1     45.982500              48.03              60.08   
           2     54.114167              56.05             118.49   
           3     54.959167              56.01             118.47   
           4     80.164583              80.02             177.66   
           5     84.503333             148.40             181.40   
...                    ...                ...                ...   
2021 4     26   102.215000             201.07             299.33   
           27    94.468333             201.03             299.09   
           28   100.343333             198.58             299.10   
           29    98.742500             195.01             296.82   
           30    98.218750             190.00             295.98   

                price_standard  off_price_mean  ...  morning_price_mean  \
year month day                                  ...                       
2016 1     1           45.8275       45.982500  ...                 NaN   
           2           50.2275       45.511333  ...                 NaN   
           3           50.6050       48.152222  ...                 NaN   
           4           80.0200       47.992222  ...                 NaN   
           5           80.0500       43.756667  ...             148.400   
...                        ...             ...  ...                 ...   
2021 4     26          70.0450       42.868889  ...             201.055   
           27          70.0200       42.197778  ...             201.015   
           28          70.0125       42.605556  ...             189.575   
           29          68.0600       42.532222  ...             187.460   
           30          67.9825       42.047778  ...             182.510   

                evening_price_mean  off_hour_count  standard_hour_count  \
year month day                                                            
2016 1     1                   NaN              24                  NaN   
           2                   NaN              15                  9.0   
           3                   NaN              18                  6.0   
           4            174.456667               9                 12.0   
           5            181.400000               9                 12.0   
...                            ...             ...                  ...   
2021 4     26           299.215000               9                 11.0   
           27           299.085000               9                 11.0   
           28           299.085000               9                 11.0   
           29           296.805000               9                 11.0   
           30           295.975000               9                 11.0   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   NaN   

In [129]:
daily.loc[2016,2,8]

volume                   3114.100000
value                  195967.843000
price_weight_av            62.929207
price_max                 171.760000
price_min                  50.360000
price_mean                 96.732917
price_morning_max         171.760000
price_evening_max         145.020000
price_standard            103.080000
off_price_mean             50.618750
standard_price_mean       109.151667
morning_price_mean        171.760000
evening_price_mean        145.020000
off_hour_count              8.000000
standard_hour_count        12.000000
morning_hour_count          1.000000
evening_hour_count          3.000000
off_volume               2333.400000
standard_volume           777.900000
morning_volume              2.800000
evening_volume              0.000000
Name: (2016, 2, 8), dtype: float64

In [130]:
daily.to_csv('dam_daily.csv')

In [131]:
dam=dam.drop(['hour_ref','month_no'],1)

In [132]:
dam.to_csv('dam_out.csv')